### Run Asset Pool

In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english',check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.8, server:0.26.5

#### Pool

In `Pool` , it will have a field named `assets` ,which contains a list of `assets`

In [9]:
mortgage1 = ["Mortgage"
            ,{"originBalance": 12000.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 40
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 10000.0
            ,"currentRate": 0.075
            ,"remainTerm": 30
            ,"status": "current"}]

mortgage2 = ["Mortgage"
            ,{"originBalance": 800.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 25
            ,"freq": "monthly"
            ,"type": "level"
            ,"originDate": "2021-02-01"}
            ,{"currentBalance": 600.0
            ,"currentRate": 0.075
            ,"remainTerm": 15
            ,"status": "current"}]

myPool = {'assets':[ mortgage1, mortgage2  ],
         'cutoffDate':"2022-03-01"}

{'HistoryDefaults': 17.77, 'HistoryDelinquency': 0, 'HistoryLoss': 17.77}

In [ ]:
r = localAPI.runPool(myPool
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
                  ,read=True)

r['PoolConsol'][0]

[('2022-03-01', 9542.79, 345.8, 61.8, 0, 7.62, 0, 7.62, 0.075, None, None, 345.8, 0, 0, 7.62, 0, 7.62),
 ('2022-04-01', 9186.98, 347.68, 59.58, 0, 8.13, 0, 8.13, 0.075, None, None, 693.48, 0, 0, 15.75, 0, 15.75),
 ('2022-05-01', 8829.85, 349.56, 57.36, 0, 7.57, 0, 7.57, 0.075, None, None, 1043.04, 0, 0, 23.32, 0, 23.32),
 ('2022-06-01', 8470.88, 351.45, 55.13, 0, 7.52, 0, 7.52, 0.075, None, None, 1394.49, 0, 0, 30.84, 0, 30.84),
 ('2022-07-01', 8110.54, 353.35, 52.89, 0, 6.99, 0, 6.99, 0.075, None, None, 1747.84, 0, 0, 37.83, 0, 37.83),
 ('2022-08-01', 7748.37, 355.26, 50.64, 0, 6.91, 0, 6.91, 0.075, None, None, 2103.1, 0, 0, 44.74, 0, 44.74),
 ('2022-09-01', 7384.6, 357.17, 48.38, 0, 6.6, 0, 6.6, 0.075, None, None, 2460.27, 0, 0, 51.34, 0, 51.34),
 ('2022-10-01', 7019.39, 359.12, 46.1, 0, 6.09, 0, 6.09, 0.075, None, None, 2819.39, 0, 0, 57.43, 0, 57.43),
 ('2022-11-01', 6652.36, 361.05, 43.82, 0, 5.98, 0, 5.98, 0.075, None, None, 3180.44, 0, 0, 63.41, 0, 63.41),
 ('2022-12-01', 6283.8

#### Cutoff day

`Cutoff day` means the day after which assets's cashflow belongs to the pool

In [13]:
myPool = {'assets':[ mortgage1, mortgage2  ],
         'cutoffDate':"2023-03-01"}

r = localAPI.runPool(myPool
                  ,poolAssump=("Pool",("Mortgage",{"CDR":0.01},None,None,None)
                                    ,None
                                    ,None)
                  ,read=True)

list(r['PoolConsol'][0].to_records())

[('2023-03-01', 5168.39, 368.94, 34.6, 0, 4.26, 0, 4.26, 0.075, None, None, 368.94, 0, 0, 4.26, 0, 4.26),
 ('2023-04-01', 4834.54, 329.45, 32.27, 0, 4.4, 0, 4.4, 0.075, None, None, 698.39, 0, 0, 8.66, 0, 8.66),
 ('2023-05-01', 4499.31, 331.24, 30.19, 0, 3.99, 0, 3.99, 0.075, None, None, 1029.63, 0, 0, 12.65, 0, 12.65),
 ('2023-06-01', 4162.45, 333.03, 28.09, 0, 3.83, 0, 3.83, 0.075, None, None, 1362.66, 0, 0, 16.48, 0, 16.48),
 ('2023-07-01', 3824.19, 334.83, 25.99, 0, 3.43, 0, 3.43, 0.075, None, None, 1697.49, 0, 0, 19.91, 0, 19.91),
 ('2023-08-01', 3484.3, 336.63, 23.88, 0, 3.26, 0, 3.26, 0.075, None, None, 2034.12, 0, 0, 23.17, 0, 23.17),
 ('2023-09-01', 3142.87, 338.46, 21.75, 0, 2.97, 0, 2.97, 0.075, None, None, 2372.58, 0, 0, 26.14, 0, 26.14),
 ('2023-10-01', 2799.99, 340.29, 19.62, 0, 2.59, 0, 2.59, 0.075, None, None, 2712.87, 0, 0, 28.73, 0, 28.73),
 ('2023-11-01', 2455.49, 342.12, 17.48, 0, 2.38, 0, 2.38, 0.075, None, None, 3054.99, 0, 0, 31.11, 0, 31.11),
 ('2023-12-01', 2109

#### Mixed Asset Pool

User can build multiple pools with different asset type

In [ ]:
receivable1 = ["Invoice"
                  ,{"start":"2024-04-01","originBalance":2000
                    ,"originAdvance":1500,"dueDate":"2024-06-01"}
                  ,{"status":"Current"}]

myMixedPool = {"invoicePool": {'assets':[receivable1],}
              ,"mortgagePool": {'assets':[mortgage1]}
              ,'cutoffDate':"2022-03-01"}

Without any stress assumption, the return result will be a map, which keys are the pool name

In [ ]:
r = localAPI.runPool(myMixedPool,read=True)

list(r['invoicePool'][0].to_records())

: 

In [ ]:
r['mortgagePool'][0]

Set assumptions to the pools
* passing a tuple with key ``ByPoolID`` to pool Assumption
* a map with key equals to the name of pool

In [ ]:
mPoolAssump = ("Pool",
               ("Mortgage",{"CDR":0.01},None,None,None)
                ,None
                ,None)

iPoolAssump = ("Pool"
                ,("Receivable", {"CDR":0.01}, None, None)
                ,None
                ,None)


r = localAPI.runPool(myMixedPool
                  ,poolAssump = ("ByPoolId",
                                    {"invoicePool":iPoolAssump
                                    ,"mortgagePool":mPoolAssump}
                                    )
                  ,read=True)

: 